Trabalho 

In [1]:
import sklearn as skl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prep5 import DataPreparation
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedShuffleSplit
from sklearn import tree

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, BaggingClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.metrics import classification_report
%matplotlib inline

In [2]:
prep = DataPreparation()

train_db = pd.read_csv("dataset/train_radiomics_hipocamp.csv")
test_db = pd.read_csv("dataset/test_radiomics_hipocamp.csv")
control_db = pd.read_csv("dataset/train_radiomics_occipital_CONTROL.csv")

train_db = prep.prep_train(train_db)
test_db = prep.transform(test_db)
control_db = prep.transform(control_db)

In [4]:
train_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 2174 entries, diagnostics_Image-original_Dimensionality to diagnostics_Mask-original_CenterOfMass_2
dtypes: float64(2173), object(1)
memory usage: 5.1+ MB


In [5]:
test_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 2173 entries, diagnostics_Image-original_Dimensionality to diagnostics_Mask-original_CenterOfMass_2
dtypes: float64(2173)
memory usage: 1.7 MB


In [6]:
X = train_db.drop(columns=["Transition"]) 
y = train_db["Transition"]  

In [7]:
X_train = pd.read_csv('prep5/train_X.csv')
X_test = pd.read_csv('prep5/test_X.csv')
y_train = pd.read_csv('prep5/train_y.csv')
y_test = pd.read_csv('prep5/test_y.csv')

In [9]:
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report

# Definindo os estimators
estimators = [
    ("gb", GradientBoostingClassifier(random_state=2024)),
    ("svm", SVC(probability=True, random_state=2024)),
    ("rf", RandomForestClassifier(random_state=2024))
]

# Modelo meta-aprendizagem
meta_learner = LogisticRegression(random_state=2024)

# StackingClassifier
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_learner,
    cv=10,  
    n_jobs=-1
)

# Definindo os hiperparâmetros para busca
param_grid = {
    'svm__C': [0.1],                    
    'svm__kernel': ['poly'],         
    'svm__degree': [3], 
    'svm__gamma': ['scale'],

    # Random Forest
    'rf__n_estimators': [50,100],        
    'rf__max_depth': [7],              
    'rf__min_samples_split': [2,5],
    'rf__min_samples_leaf': [2,5],
    'rf__max_features': ['log2'],

    # Gradient Boosting
    'gb__n_estimators': [50,100],
    'gb__learning_rate': [0.1],
    'gb__max_depth': [5],
    'gb__min_samples_split': [5],
    'gb__min_samples_leaf': [2],

    'final_estimator__C': [0.1,1],
}

# Estratégia de validação cruzada
cv_strategy = StratifiedKFold(n_splits=10)

# GridSearchCV para o StackingClassifier
grid_search = GridSearchCV(
    estimator=stacking_clf,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=cv_strategy,
    verbose=1,
    refit=True,
    n_jobs=-1
)

# Ajustando o modelo
grid_search.fit(X, y)

# Melhor conjunto de hiperparâmetros
print("Melhores parâmetros encontrados:", grid_search.best_params_)

# Melhor modelo treinado
best_model = grid_search.best_estimator_

# Validação cruzada com o melhor modelo
print("F1 Score médio com validação cruzada:", grid_search.best_score_)

# Avaliação no conjunto de teste
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Matriz de confusão e relatório de classificação
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:")
print(conf_matrix)

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

Fitting 10 folds for each of 4 candidates, totalling 40 fits


/home/tiagogr/miniconda3/envs/DAA/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/home/tiagogr/miniconda3/envs/DAA/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/home/tiagogr/miniconda3/envs/DAA/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/home/tiagogr/miniconda3/envs/DAA/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/home/tiagogr/miniconda3/envs/DAA/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class

Melhores parâmetros encontrados: {'dt__max_depth': 10, 'dt__min_samples_split': 5, 'final_estimator__C': 1, 'rf__max_depth': 10, 'rf__n_estimators': 100, 'svm__C': 0.1, 'svm__kernel': 'linear'}


/home/tiagogr/miniconda3/envs/DAA/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/home/tiagogr/miniconda3/envs/DAA/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/home/tiagogr/miniconda3/envs/DAA/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/home/tiagogr/miniconda3/envs/DAA/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/home/tiagogr/miniconda3/envs/DAA/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class

F1 Score médio com validação cruzada: 0.3445051712964552


/home/tiagogr/miniconda3/envs/DAA/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tiagogr/miniconda3/envs/DAA/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Matriz de Confusão:
[[15  0  0  3  2]
 [ 1 12  0  1  5]
 [ 0  0 19  0  0]
 [ 5  1  0  8  5]
 [ 0  7  0  0 12]]

Relatório de Classificação:
              precision    recall  f1-score   support

       AD-AD       0.71      0.75      0.73        20
       CN-CN       0.60      0.63      0.62        19
      CN-MCI       1.00      1.00      1.00        19
      MCI-AD       0.67      0.42      0.52        19
     MCI-MCI       0.50      0.63      0.56        19

    accuracy                           0.69        96
   macro avg       0.70      0.69      0.68        96
weighted avg       0.70      0.69      0.68        96



# Testar Controlo

In [ ]:
CX = control_db.drop(columns=["Transition"]) 
Cy = control_db["Transition"]  

control_pred = best_model.predict(CX)

control_f1_score = f1_score(Cy, control_pred, average='macro')
print("F1-Score no conjunto de controle:", control_f1_score)

# Matriz de Confusão
ConfusionMatrixDisplay.from_predictions(Cy, control_pred, cmap='Blues')
plt.show()

# Relatório de Classificação
print("\nRelatório de Classificação no conjunto de controle:")
print(classification_report(Cy, control_pred))


### Guardar o melhor modelo do stacking

In [11]:
best_model.fit(X, y)
test_predictions = best_model.predict(test_db)

row_ids = range(1, len(test_predictions) + 1)

output_df = pd.DataFrame({
    "RowId": row_ids,
    "Result": test_predictions
})


output_df.to_csv("resultados/stck5.csv", index=False)